# Finger People experimental playground

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import math
import urllib
import time
from enum import Enum
import subprocess
import keras.models
import pickle

from helpers import imshow
from capture import Capture, CapType
from segmenter import *
from gestures import *
from game_input import GameInput
import vision
import dataset
import learn
from cnn_input import CnnInput
import preprocess
import finger_people

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Input Sources

In [ ]:
# IP Webcam
cap_source = 'http://192.168.0.124:8080/video'
cap_type = CapType.VIDEO

# Learning Approach

## Record Data
Record dense data:

In [ ]:
gesture_name = 'run'
max_frames = 300
n_frames = 0
output_dir = '../data/'

output_path = os.path.join(output_dir,
        '{}-{}.avi'.format(gesture_name, str(time.time())))

cap = Capture(cap_source, cap_type)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (160,120))

while cap.is_opened() and n_frames < max_frames:
    ret, frame = cap.read()
    if ret == False:
        break
        
    out.write(frame)
    n_frames += 1

    cv2.imshow('frame', frame)

    key = cv2.waitKey(3) & 0xFF
    if key == ord('q'):
        break
            
cap.kill()

Record sparse data:

In [ ]:
gesture_name = 'kick'
max_frames = 300
n_frames = 0
recorded_frames = set()
output_dir = '../data/'
output_prefix = os.path.join(output_dir,
        '{}-{}'.format(gesture_name, str(time.time())))
output_path = output_prefix + '.avi'
output_pickle = output_prefix + '.pickle'

cap = Capture(cap_source, cap_type)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (160,120))
flow = vision.OpticalFlow(cap.read()[1])
record = False

while cap.is_opened() and len(recorded_frames) < max_frames:
    ret, frame = cap.read()
    if ret == False:
        break
    
    out.write(frame)
    flow.update(frame)
    
    if record:
        recorded_frames.add(n_frames)        
        cv2.circle(frame, (6,6), (5), (0,0,255), cv2.FILLED)
    n_frames += 1
        
    cv2.imshow('frame', np.hstack((frame, flow.vis)))
    
    key = cv2.waitKey(3) & 0xFF
    if key == ord('q'):
        break
    if key == ord('f'):
        record = not record
            
pickle.dump(recorded_frames, open(output_pickle, 'wb'))
cap.kill()

## Preprocess Data

In [1]:
preprocess.preprocess()

NameError: name 'preprocess' is not defined

## Train Model

In [ ]:
learn.train()

## Load Model

In [ ]:
model_name = '4_long_train.hdf5'
model_dir = 'models'
model_path = os.path.join(model_dir, model_name)

model = keras.models.load_model(model_path)

Test how fast our model runs while also doing a basic sanity check for accuracy.

In [ ]:
path = '../data/preprocessed/walk/{}.png'
predictions = np.zeros(dataset.n_classes)
start = time.time()
n = 600
for i in range(n):
    frame = cv2.imread(path.format(i))
    frame = np.expand_dims(frame, 0)
    prediction = model.predict(frame)
    label_id = np.argmax(prediction)
    predictions[label_id] = predictions[label_id] + 1

label_predictions = {}
for i, prediction in enumerate(predictions):
    label_predictions[dataset.id_to_gesture[i]] = predictions[i]
print(label_predictions)

rate = n / (time.time() - start)
print('{:.2f} forward passes per second.'.format(rate))

## Finger People

In [ ]:
finger_people.finger_people(model_path, cap_source, cap_type)

In [ ]:
# In case of emergency
cap.kill()

# Vision Approach

## Skin Clibration

In [ ]:
HUE = 0
SAT = 1
VAL = 2
channels = [HUE, SAT]

HUE_RANGE = (0,180)
SAT_RANGE = (0,256)
VAL_RANGE = (0, 256)
ranges = [*HUE_RANGE, *SAT_RANGE]

num_bins = 32
bins = [num_bins]*len(channels)

sat_thresh = 16 # any saturations below will be thrown out due to instability
sat_thresh_bin = int(sat_thresh/SAT_RANGE[1]*num_bins)

# Get samples of skin
cap = Capture(cap_source, cap_type)
skin_samples = []
for _ in range(5):
    skin_samples.append(cv2.cvtColor(get_roi_sample(cap), cv2.COLOR_BGR2HSV))
cap.kill()

# Calculate histogram
skin_hist = mean_hist(skin_samples, channels=channels, ranges=ranges, bins=bins)
#skin_hist[:, :sat_thresh_bin] = 0
plt.xlabel('Saturation')
plt.ylabel('Hue')
plt.title('Your Skin')
plt.imshow(skin_hist)

# Create function to mask skin
mask_skin = lambda frame, thresh=1: hist_mask(frame, skin_hist, thresh=thresh, channels=channels, ranges=ranges)

## Finger People

In [ ]:
cap = Capture(cap_source, cap_type)

hand = Hand()

walks_left = True # The direction the finger person is facing in the (mirrored) frame
if not walks_left:
    hand.body_facing = Direction.RIGHT
    
input_toggle = False

min_blob_size = 0
mhi_alpha = 0.5

# Take the first frame
_ret, init_frame = cap.read()
prvs = cv2.cvtColor(init_frame,cv2.COLOR_BGR2GRAY)
frame1 = cv2.flip(init_frame, 1)

h = init_frame.shape[0]
w = init_frame.shape[1]
hand.screen_width = w

# For color representation of optical flow
flow_vis = np.zeros_like(init_frame)
flow_vis[...,1] = 255

# Motion history image
mag_hist = np.zeros((init_frame.shape[:2]), dtype=np.float32)
ang_hist = np.zeros((init_frame.shape[:2]), dtype=np.float32)
mhi = np.zeros_like(init_frame)
mhi[...,1] = 255

while cap.is_opened():
    
    # Get the next frame
    _ret, frame = cap.read()
    frame = cv2.flip(frame, 1)  
    if not _ret:
        break
    debug = frame.copy()
      
    
    
    ''' SKIN MASK '''
    blurred = cv2.GaussianBlur(frame, (7,7), 0)
    skin_mask = mask_skin(cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV), thresh=25)
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    hand_mask = skin_mask
    
    hand_mask = cv2.morphologyEx(hand_mask, cv2.MORPH_OPEN, kernel, iterations = 1)
    hand_mask = cv2.morphologyEx(hand_mask, cv2.MORPH_CLOSE, kernel, iterations = 2)
    hand_contour = largest_blob(hand_mask, thresh=min_blob_size)
    hand_mask = contour2mask(hand_contour, shape=skin_mask)
    
    hand.position = contour_pos(hand_contour)
    
    # debug
    if hand.position[0] is not None:
        cv2.putText(debug, '{:.2f},{:.2f}'.format(hand.velocity[0], hand.velocity[1]), (0, 50), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,255,0))
        cv2.circle(debug, (int(hand.position[0]), int(hand.position[1])), 4, (255,0,0), -1)
        
        
        
    ''' KEY POINTS/REGIONS '''
    if hand_contour is not None:
        most_left  = tuple(hand_contour[hand_contour[:, :, 0].argmin()][0])
        most_right = tuple(hand_contour[hand_contour[:, :, 0].argmax()][0])
        most_down  = tuple(hand_contour[hand_contour[:, :, 1].argmax()][0])

        eoh = most_left if walks_left else most_right # end of hand
        tl = (int(eoh[0]), int(hand.position[1]))
        br = (int(hand.position[0]), h-1)
        legs_roi = (tl, br) 

        # Debug
        cv2.circle(debug, most_left, 4, (255,0,255), -1)
        cv2.circle(debug, most_down, 4, (0,255,0), -1)
        cv2.rectangle(debug, tl, br, (0,0,255), 2)

    
    
    ''' EDGES '''
#     edges = cv2.cvtColor(blurred, cv2.COLOR_BGR2GRAY)
#     edges = cv2.adaptiveThreshold(edges, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
#     canny = cv2.Canny(blurred, 10, 10)
    
    
        
    ''' OPTICAL FLOW '''
    curr = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvs, curr, None, **fb_params)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    
    p0 = legs_roi[0]
    p1 = legs_roi[1]
    legs_mag = mag[p0[1]:p1[1], p0[0]:p1[0]]#*np.abs(np.cos(ang[p0[1]:p1[1], p0[0]:p1[0]]))
    hand.leg_speed = np.mean(legs_mag)
    
    cv2.putText(debug, '{:.2f}'.format(np.mean(legs_mag)), (0, 30), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,255,0))
    
    # Ignore non-hand movement
    #mag = cv2.bitwise_and(mag, mag, mask=hand_mask)    
       
        
        
    ''' MHI '''
#     mag_hist = mhi_alpha*mag_hist + (1-mhi_alpha)*mag
#     ang_hist = mhi_alpha*ang_hist + (1-mhi_alpha)*ang
#     mhi[...,0] = ang_hist*180/np.pi/2
#     mhi[...,2] = cv2.normalize(mag_hist, None, 0, 255, cv2.NORM_MINMAX)
    
    
    
    ''' GESTURES '''
    hand.check_cooldowns()
    cv2.putText(debug, hand.gestures_pretty() + hand.r_direction.value, (0, h), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,255,0))
    
    ''' INPUT '''
    if input_toggle:
        if Gesture.JUMP in hand.gestures:
            GameInput.jump()
        else:
            GameInput.stop_jump()

        if Gesture.RUN in hand.gestures:
            GameInput.walk(hand.r_direction)
        else:
            GameInput.stop_move()
    
    ''' OUTPUT/DEBUG '''
    hand_mask = cv2.cvtColor(hand_mask, cv2.COLOR_GRAY2BGR)
#     edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
#     canny = cv2.cvtColor(canny, cv2.COLOR_GRAY2BGR)
    
    # Create color representation of optical flow
    flow_vis[...,0] = ang*180/np.pi/2
    flow_vis[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    flow_vis_bgr = cv2.cvtColor(flow_vis,cv2.COLOR_HSV2BGR)
    
    # Create color representation of mhi
    mhi_bgr = cv2.cvtColor(mhi, cv2.COLOR_HSV2BGR)
    
    cv2.imshow('frame',np.hstack((debug, cv2.cvtColor(skin_mask, cv2.COLOR_GRAY2BGR), hand_mask, flow_vis_bgr))) #flow_vis_bgr, mhi_bgr)))

    # Exit on ESC
    key = cv2.waitKey(3) & 0xFF
    if key == 27:
        break
    if key == ord('q'):
        break
    if key == ord(' '):
        input_toggle = not input_toggle
    if key == ord('s'):
        cv2.imwrite('{}.png'.format(str(time.time())), np.hstack((debug, cv2.cvtColor(skin_mask, cv2.COLOR_GRAY2BGR), hand_mask, flow_vis_bgr)))

    prvs = curr
        
cap.kill()


In [ ]:
# In case of emergency
cap.kill()